In [1]:
import requests
import urllib
import html2text as ht
import re
import pandas as pd
import numpy as np
import json
import string
import traceback
from bs4 import BeautifulSoup
from collections import Counter

## Scrape Billboard for song titles

In [2]:
class DateTracker:
    
    days = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 
            7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    
    def __init__(self, year = None, month = None, day = None):
        self.year = year
        self.month = month
        self.day = day
        
    def previous_week(self):
        
        if (self.year == 1958) and (self.month == 8) and (0 < self.day <= 7):
            return
        
        # if move to previous month
        if self.day - 7 <= 0:
            
            if self.month != 1:
                
                self.month -= 1
                self.day += self.days[self.month] - 7
            
            # if move to previous year
            else:
                
                self.year -= 1
                self.month = 12
                self.day += self.days[self.month] - 7
                
        else:
            
            self.day -= 7
            
    def as_date(self):
        
        return "%04d-%02d-%02d" % (self.year, self.month, self.day)
        
        
class Song:
    
    def __init__(self, artist, name, rank, peakpos, weeksonchart, date):
        self.artist = artist
        self.name = name
        self.rank = rank
        self.peakpos = peakpos
        self.weeksonchart = weeksonchart
        self.date = date
        
    def get_artist(self):
        return self.artist
    
    def get_name(self):
        return self.name
    
    def get_rank(self):
        return self.rank
    
    def get_peak_pos(self):
        return self.peakpos
    
    def get_weeks_on_chart(self):
        return self.weeksonchart
    
    def get_date(self):
        return self.date
    
    def list_form(self):
        return [self.artist, self.name, self.rank, self.peakpos,
                self.weeksonchart, self.date]
    
    def summary(self):
        print("\n".join([str(x) for x in [self.artist, self.name,
                                          self.rank, self.peakpos,
                                          self.weeksonchart, self.date]]))

In [3]:
def clean_text(text):
    bad = {"\\\\n": "\n", 
           "\n+": "\n", 
           "\n": " - ",
           "\[\]\(.{0,}?\)": "",
           "\[Play\]\(.{0,}?\)": "",
           "\[\s{0,5}-\s{0,5}Song\s{0,5}-\s{0,5}Lyrics\s{0,5}-\s{0,5}\]\(.{0,}?\)": ""}

    for x in bad:
        text = re.sub(x, bad[x], text, flags = re.I | re.S)
        
    return text

def clean_line(text):
    repl = {"weeks at no. 1": "",
            "(-\s{1,4}!)+": "",
            "(?<![A-z])! -": "",
            "_": "",
            "\[\s-": "", 
            "-\s\]": "",
            "\[.{0,}?\]|\(.{0,}?\)": "",
            "\s+": " ",
            "(-\s{1,5})+": "- ",}    
    
    for x in repl:
        text = re.sub(x, repl[x], text, flags = re.I | re.S)
    return text

In [4]:
def get_songs_by_week(weeks = 100):
    
    #filler = [" and ", " featuring ", " & ", " x ", " / "]
    namesub = {"f\*\*k": "fuck", "s\*\*t": "shit"}
    path = "https://www.billboard.com/charts/hot-100"
    curdate = DateTracker(year = 2018, month = 8, day = 11)
    songs = []

    for j in range(weeks):
        
        page = requests.get(path + "/" + curdate.as_date())
        if j % 20 == 0:
            print("We are on week {}, it is {}".format(j, curdate.as_date()))

        data = ht.html2text(str(page.content))
        data = clean_text(data)
        data = re.split("date search |in performance - |on chart - ", data, flags = re.I)[1:]

        #first one
        weeksoc = int(re.search("[0-9]{0,1}\s-\sweeks", 
                                re.sub("weeks at no. 1", "", data[0], flags = re.I | re.S),
                                flags = re.I | re.S).group()[:2])
        tmp = clean_line(data[1]).strip().split("-")
        name = tmp[0].strip()
        for k in namesub:
            name = re.sub(k, namesub[k], name, flags = re.I | re.S)
        artist = tmp[1].strip()
        # for k in filler:
        #     artist = re.sub(k, ", ", artist, flags = re.I | re.S)

        songs.append(Song(artist = artist, name = name, rank = 1,
                          peakpos = 1, weeksonchart = weeksoc, 
                          date = curdate.as_date()))


        #everything else (2 - 100)
        for i in data[2:]:
            #we know we have a rank
            if re.match("^[0-9]{0,3}", i.strip()).group() != "":
                i = clean_line(i)
                tmp = i.split("-")
                rank = tmp[0].strip()
                name = tmp[1].strip()
                for k in namesub:
                    name = re.sub(k, namesub[k], name, flags = re.I | re.S)
                artist = tmp[2].strip()
                # for k in filler:
                #     artist = re.sub(k, ", ", artist, flags = re.I | re.S)
                    
                #if these are empty they're debuts!
                try:
                    peakpos = int(re.search("[0-9]{0,3}\s-\speak position", i, flags = re.I | re.S).group().split(" ")[0])
                except AttributeError:
                    peakpos = ""
                try:
                    weeksoc = int(re.search("[0-9]{0,3}\s-\sweeks", i, flags = re.I | re.S).group().split(" ")[0])
                except AttributeError:
                    weeksoc = ""

                songs.append(Song(artist = artist, name = name, rank = rank,
                                  peakpos = peakpos, weeksonchart = weeksoc,
                                  date = curdate.as_date()))

        curdate.previous_week()
        
    return songs, curdate

In [5]:
songs, curdate = get_songs_by_week(weeks = 200)

We are on week 0, it is 2018-08-11
We are on week 20, it is 2018-03-24
We are on week 40, it is 2017-11-04
We are on week 60, it is 2017-06-17
We are on week 80, it is 2017-01-28
We are on week 100, it is 2016-09-10
We are on week 120, it is 2016-04-23
We are on week 140, it is 2015-12-04
We are on week 160, it is 2015-07-17
We are on week 180, it is 2015-02-27


In [6]:
prep = np.array([x.list_form() for x in songs])
df = pd.DataFrame({
    "Artists": prep[:, 0],
    "Name": prep[:, 1],
    "Weekly rank": prep[:, 2],
    "Peak position": prep[:, 3],
    "Weeks on chart": prep[:, 4],
    "Week": prep[:, 5]
})

In [7]:
df.to_csv("Weekly_ranks.csv", header = True, index = False)

## Scrape Genius for lyrics, genre, etc.


Song name

Artist/Group

Weekly rank 

Peak rank

Year published

Genre

Writing Credits

Lyrics



In [264]:
df.iloc[97]

Artists           YG Featuring A$AP Rocky
Name                              Handgun
Peak position                            
Week                           2018-08-11
Weekly rank                            99
Weeks on chart                           
Name: 97, dtype: object

In [ ]:
re.search("A\$aP")

In [279]:
re.search("A\$AP", "HaNDGUN by YG (Ft. a$aP Rocky)", flags = re.I | re.S)

<_sre.SRE_Match object; span=(19, 23), match='a$aP'>

In [298]:
get_lyric_link(df.iloc[97]["Artists"], df.iloc[97]["Name"], debug = True)

['YG', 'A\\$AP', 'Rocky', 'Handgun']
HaNDGUN by YG (Ft. a$aP Rocky)


'/Yg-handgun-lyrics'

In [9]:
def get_lyric_link(artists, name, both = False, debug = False):
    
    """
    Returns the link of the song name given artists + name of the song
    """
    
    client_token = "BrGsH3KoiMzSyCUClF4-TyzjrNfQfrr2-Q9bfK6Bhum1fquRgVf0rn-Pq6mr9Uyc"
    headers = {"Authorization": "Bearer " + client_token}
    link = "https://api.genius.com/search?q="
    space = "%20"
    
    #this could prove disastrous, double check it...
    # update: yes it did, just replace accents.
    name_repl = {"Beyonce": "Beyoncé", 
                 "Amine": "Aminé",
                 "D.R.A.M.": "DRAM",
                 "$ign": "\$ign"}
    repl = {
        "a|á|ạ|à|ả|ã|ă|ắ|ặ|ằ|ẳ|ẵ|â|ấ|ậ|ầ|ẩ|ẫ": "a",
        "é|ẹ|è|ẻ|ẽ|ê|ế|ệ|ề|ể|ễ|ë": "e",
        "í|ị|ì|ỉ|ĩ": "i",
        "ó|ọ|ò|ỏ|õ|ô|ố|ộ|ồ|ổ|ỗ|ơ|ớ|ợ|ờ|ỡ": "o",
        "ú|ụ|ù|ủ|ũ|ư|ứ|ự|ừ|ử|ữ": "u",
        "ý|ỵ|ỳ|ỷ|ỹ": "y"
    }
    artistregex = {
        "\$": "\\\\$"
    }
    filler = [" and ", " featuring ", " & ", " x ", " / "]

    artiststmp = re.sub(",", "", artists)
    name = re.sub(",", "", name)
    if both:
        page = requests.get(link + re.sub(" ", space, name) +
                            space + re.sub(" ", space, artiststmp), headers = headers)
    else:
        page = requests.get(link + re.sub(" ", space, name), headers = headers)
        
    # now that we searched, remove filler words that may not appear
    # in actual song title
    page = json.loads(page.content)["response"]["hits"]
    for i in filler:
        artiststmp = re.sub(i, " ", artiststmp, flags = re.I | re.S)
    check = [re.sub(",", "", x) for x in artiststmp.split(" ") + name.split(" ") if x not in string.punctuation]


    # fix artist tokens to be used in re.search
    for i in artistregex:
        for j in range(len(check)):
            check[j] = re.sub(i, artistregex[i], check[j], flags = re.I | re.S)
            
    if debug:
        print(check)    
        
    top = []

    
    
    if len(page) == 1:
        return page[0]["result"]["path"]
    else:
        for i in range(len(page)):
            c = 0
            
            # remove accents from title
            title = page[i]["result"]["full_title"]
            for j in repl:
                title = re.sub(j, repl[j], title, flags = re.I | re.S)

            if debug:
                print(title)
                
            # check if every artist + name token in the full title
            for j in check:
                if re.search(j, title, flags = re.I | re.S) != None:
                    c += 1
            if c == len(check):
                try:
                    if page[i]["result"]["stats"]["pageviews"] > 0:
                        return page[i]["result"]["path"]
                except KeyError:
                    continue

        
    #if we are here, search failed. now we include the artists as well
    if both == False:
        return get_lyric_link(artiststmp, name, True)
    
    top = []
    
    #if here, find most popular song
    print()
    print("hopefully we never see this message.")
    print(artiststmp, name)
    for j in range(len(page)):
        try:
            top.append(page[i]["result"]["stats"]["pageviews"])
        except Exception:
            top.append(-1)
    print(page[np.argmax(top)]["result"]["path"])
    print()
    return page[np.argmax(top)]["result"]["path"]





def scrape_lyrics(row, snip):
    
    # fix artist names w/ these
    repl = {
        "á|ạ|à|ả|ã|ă|ắ|ặ|ằ|ẳ|ẵ|â|ấ|ậ|ầ|ẩ|ẫ": "a",
        "é|ẹ|è|ẻ|ẽ|ê|ế|ệ|ề|ể|ễ|ë": "e",
        "í|ị|ì|ỉ|ĩ": "i",
        "ó|ọ|ò|ỏ|õ|ô|ố|ộ|ồ|ổ|ỗ|ơ|ớ|ợ|ờ|ỡ": "o",
        "ú|ụ|ù|ủ|ũ|ư|ứ|ự|ừ|ử|ữ": "u",
        "ý|ỵ|ỳ|ỷ|ỹ": "y",
        "\$": "\\\\$"
    }
    
    #print(snip)
    base_link = "https://genius.com"
    content = requests.get(base_link + snip).content
    data = BeautifulSoup(content, "lxml").get_text()
    
    ##########################################
    
    # get all writers (expensive search)
    
    meta_data = content.decode("utf8")
    
    start_meta_regex = "verified_lyrics_by.*?writer_artists&quot"
    stop_meta_regex = "itemprop=\"page_data\""
    write_regex = "https://genius.com/artists/[^&]*"

    start = re.search(start_meta_regex, meta_data, flags = re.I | re.S).end()
    stop = re.search(stop_meta_regex, meta_data, flags = re.I | re.S).end()
    meta_data = meta_data[start:stop]

    write_links = re.findall(write_regex, meta_data)
    write = [re.sub("https://genius.com/artists/", "", x) for x in write_links]
    write = ", ".join([re.sub("-", " ", x) for x in write])
    
    
    ##########################################
    
    #subset data to search for lyrics
    
    genre_sub = {"\"": " ", "genius": "", "\s+,": ","}
    lyric_regex = row["Name"] + ".{0,20}?Lyrics.+?More on Genius"
    genre_regex = "genres\":\[.+?\]"
    track_info_regex = "\"{}".format(row["Name"]) + ".{0,20}?\".{0,20}?track info.+?remixed by"
    track_info_regex2 = "\"{}".format(row["Name"]) + ".{0,20}?\".{0,20}?track info.+?cover by"
    date_regex = "release date.{0,15}?20[0-9][0-9]"
    write_regex = "Written By\n+.+?\n"
    
    data = data[re.search(base_link + snip, data).end():]
    for i in repl:
        data = re.sub(i, repl[i], data, flags = re.I | re.S)
    
    lyrics = re.search(lyric_regex, data, flags = re.I | re.S).group()[:-15]
    
    try:
        genre = re.search("\[.+?\]", re.search(genre_regex, data, flags = re.I | re.S).group()).group()[1:-1]
        for k in genre_sub:
            genre = re.sub(k, genre_sub[k], genre, flags = re.I)
    except AttributeError:
        genre = ""
    
        
    try:
        date = re.split("\n+", re.search(date_regex, data, flags = re.I | re.S).group())[-1]
    except AttributeError:
        date = ""
        
    return date, genre, write, lyrics

In [10]:
dates = []
genre = []
write = []
lyrics = []
checker = {}

for i in range(0, len(df)):
    
    art = df.iloc[i]["Artists"]
    name = df.iloc[i]["Name"]
    
    # haven't scraped the song lyrics yet
    
    if art + name not in checker:
        try:
            
            temp = get_lyric_link(art, name)
            checker.update({art + name: i})
            d, g, w, l = scrape_lyrics(df.iloc[i], temp)
            
        except Exception:
            
            #rough hack to just skip for now
            #that way we can view all the errors in one go!!
            #i need to sleep...
            print(i)
            traceback.print_exc()
            dates.append("")
            genre.append("")
            write.append("")
            lyrics.append("")
            continue
            
        dates.append(d)
        genre.append(g)
        write.append(w)
        lyrics.append(l)
    
    # already scraped song lyrics
    
    else:
        
        num = checker[art + name]
        d, g, w, l = dates[num], genre[num], write[num], lyrics[num]
        
        dates.append(d)
        genre.append(g)
        write.append(w)
        lyrics.append(l)        
    
    if i % 500 == 0:
        print("We are {}% done".format(round(i / len(df) * 100, 4)))

We are 0.0% done

hopefully we never see this message.
TK Kravitz Jacqueez Ocean
/Tk-kravitz-ocean-lyrics


hopefully we never see this message.
Mitchell Tenpenny Drunk Me
/Mitchell-tenpenny-drunk-me-lyrics


hopefully we never see this message.
Iggy Azalea Feauring Tyga Kream
391


Traceback (most recent call last):
  File "C:\Users\Sam\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py", line 57, in _wrapfunc
    return getattr(obj, method)(*args, **kwds)
AttributeError: 'list' object has no attribute 'argmax'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-10-5e9c678adc8b>", line 17, in <module>
    temp = get_lyric_link(art, name)
  File "<ipython-input-9-90b549fde30e>", line 87, in get_lyric_link
    return get_lyric_link(artiststmp, name, True)
  File "<ipython-input-9-90b549fde30e>", line 100, in get_lyric_link
    print(page[np.argmax(top)]["result"]["path"])
  File "C:\Users\Sam\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py", line 963, in argmax
    return _wrapfunc(a, 'argmax', axis=axis, out=out)
  File "C:\Users\Sam\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py", line 67, in _wrapfunc
    return _wrapit(obj, method, *args, **kwds)
  File "C:\Users\Sam\Ana

492


Traceback (most recent call last):
  File "<ipython-input-10-5e9c678adc8b>", line 19, in <module>
    d, g, w, l = scrape_lyrics(df.iloc[i], temp)
  File "<ipython-input-9-90b549fde30e>", line 161, in scrape_lyrics
    lyrics = re.search(lyric_regex, data, flags = re.I | re.S).group()[:-15]
AttributeError: 'NoneType' object has no attribute 'group'


We are 2.5253% done
553


...

Traceback (most recent call last):
  File "<ipython-input-10-5e9c678adc8b>", line 17, in <module>
    temp = get_lyric_link(art, name)
  File "<ipython-input-9-90b549fde30e>", line 75, in get_lyric_link
    if re.search(j, title, flags = re.I | re.S) != None:
  File "C:\Users\Sam\Anaconda3\lib\re.py", line 182, in search
    return _compile(pattern, flags).search(string)
  File "C:\Users\Sam\Anaconda3\lib\re.py", line 301, in _compile
    p = sre_compile.compile(pattern, flags)
  File "C:\Users\Sam\Anaconda3\lib\sre_compile.py", line 562, in compile
    p = sre_parse.parse(p, flags)
  File "C:\Users\Sam\Anaconda3\lib\sre_parse.py", line 855, in parse
    p = _parse_sub(source, pattern, flags & SRE_FLAG_VERBOSE, 0)
  File "C:\Users\Sam\Anaconda3\lib\sre_parse.py", line 416, in _parse_sub
    not nested and not items))
  File "C:\Users\Sam\Anaconda3\lib\sre_parse.py", line 616, in _parse
    source.tell() - here + len(this))
sre_constants.error: nothing to repeat at position 0



hopefully we never see this message.
The Swon Brothers Later On
/The-swon-brothers-later-on-lyrics


hopefully we never see this message.
Miranda Lambert Duet With Carrie Underwood Somethin' Bad
/Country-music-awards-cma-2014-nominees-lyrics

19759


Traceback (most recent call last):
  File "<ipython-input-10-5e9c678adc8b>", line 19, in <module>
    d, g, w, l = scrape_lyrics(df.iloc[i], temp)
  File "<ipython-input-9-90b549fde30e>", line 161, in scrape_lyrics
    lyrics = re.search(lyric_regex, data, flags = re.I | re.S).group()[:-15]
AttributeError: 'NoneType' object has no attribute 'group'



hopefully we never see this message.
Florida Georgia Line Bumpin' The Night
/Florida-georgia-line-bumpin-the-night-lyrics

19784


Traceback (most recent call last):
  File "<ipython-input-10-5e9c678adc8b>", line 17, in <module>
    temp = get_lyric_link(art, name)
  File "<ipython-input-9-90b549fde30e>", line 75, in get_lyric_link
    if re.search(j, title, flags = re.I | re.S) != None:
  File "C:\Users\Sam\Anaconda3\lib\re.py", line 182, in search
    return _compile(pattern, flags).search(string)
  File "C:\Users\Sam\Anaconda3\lib\re.py", line 301, in _compile
    p = sre_compile.compile(pattern, flags)
  File "C:\Users\Sam\Anaconda3\lib\sre_compile.py", line 562, in compile
    p = sre_parse.parse(p, flags)
  File "C:\Users\Sam\Anaconda3\lib\sre_parse.py", line 855, in parse
    p = _parse_sub(source, pattern, flags & SRE_FLAG_VERBOSE, 0)
  File "C:\Users\Sam\Anaconda3\lib\sre_parse.py", line 416, in _parse_sub
    not nested and not items))
  File "C:\Users\Sam\Anaconda3\lib\sre_parse.py", line 616, in _parse
    source.tell() - here + len(this))
sre_constants.error: nothing to repeat at position 0


In [14]:
sum(True for x in lyrics if x == "")/len(lyrics)

0.050101010101010104

In [12]:
df["Date"] = dates
df["Genre"] = genre
df["Writing Credits"] = write
df["Lyrics"] = lyrics
df.to_csv("Weekly_data.csv", header = True, index = False)

In [310]:
df.iloc[108]

Artists                  Ariana Grande
Name              No Tears Left To Cry
Peak position                        3
Week                        2018-08-04
Weekly rank                         11
Weeks on chart                      14
Name: 108, dtype: object

Debug log:
 - Te Bote: accent error
 - Tiesto: accent error
 - Logic, Ryan Tedder: just checked hot links

In [184]:
base_link = "https://genius.com"
content = requests.get(base_link + "/Nio-garcia-casper-magico-and-bad-bunny-te-bote-remix-lyrics").content
data = BeautifulSoup(content, "lxml").get_text()

In [259]:
df.iloc[97]

Artists           YG Featuring A$AP Rocky
Name                              Handgun
Peak position                            
Week                           2018-08-11
Weekly rank                            99
Weeks on chart                           
Name: 97, dtype: object